In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

class ARIAModel(tf.keras.Model):
    def __init__(self):
        super(ARIAModel, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3))
        self.pool1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.conv2 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')
        self.pool2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(128, activation='relu')
        self.output_layer = tf.keras.layers.Dense(1, activation='sigmoid')  # Binary classification

    def call(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.dense1(x)
        return self.output_layer(x)

def train_aria(data, labels, epochs=50, batch_size=32):
    model = ARIAModel()
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(data, labels, epochs=epochs, batch_size=batch_size, validation_split=0.2)
    return model

def preprocess_images(image_paths, labels):
    data = []
    for img_path in image_paths:
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=(128, 128))
        img = tf.keras.preprocessing.image.img_to_array(img)
        data.append(img)
    return np.array(data), np.array(labels)

def main():
    # Load your dataset
    df = pd.read_csv('radiology_data.csv')  # Your dataset path
    image_paths = df['image_path'].values
    labels = df['label'].values  # Binary labels (0 or 1)
    
    # Preprocess images
    data, labels = preprocess_images(image_paths, labels)

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

    # Train the model
    model = train_aria(X_train, y_train, epochs=50, batch_size=32)

    # Evaluate the model
    y_pred = (model.predict(X_test) > 0.5).astype("int32")
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

if __name__ == "__main__":
    main()